In [1]:
import panel as pn
import hvplot.pandas
import holoviews as hv
import xarray as xr
import pandas as pd
import datetime as dt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
#import shutup
from datetime import date
from datetime import datetime
#shutup.please()
hv.extension('plotly')
pn.extension('tabulator', sizing_mode="stretch_width")

import ipywidgets as ipw
import hvplot.xarray # noqa
import panel.widgets as pnw


In [2]:
#Importing table and cleaning it into a usable dataframe
data = xr.open_dataset('Model_Percent_Change.nc')

table = data.to_dataframe().reset_index()
table = table.loc[:, ~table.columns.duplicated()].copy()
table = table.drop(columns = ['lat', 'lon', 'quantile'], axis = 1)
table.head(5)

,City,model,tmean_change_ssp126,tmean_change_ssp370,tmax_change_ssp126,tmax_change_ssp370,tmin_change_ssp126,tmin_change_ssp370,pr_change_ssp126,pr_change_ssp370,heavy_pr_change_ssp126,heavy_pr_change_ssp370,wind_change_ssp126,wind_change_ssp370
0,Birmingham,0,0.034107,0.048759,0.040029,0.047211,0.028131,0.051326,0.049144,0.003135,0.051938,0.068210,-0.052476,-0.001333
1,Birmingham,1,0.034063,0.055996,0.024383,0.041312,0.045649,0.075212,0.132690,0.136843,0.131555,0.180323,-0.058690,-0.021790
2,Birmingham,2,0.033612,0.048692,0.024959,0.040017,0.042644,0.057996,0.067448,0.061908,0.079299,0.087733,-0.095725,-0.006479
3,Birmingham,3,0.046931,0.054725,0.041735,0.044906,0.052287,0.064797,0.113668,0.105649,0.073515,0.085906,-0.004911,-0.009812
4,Birmingham,4,0.060397,0.090188,0.048458,0.074669,0.071014,0.106049,0.087416,0.066285,0.114084,0.128322,-0.058213,-0.047573


In [3]:
#Filters table for the greatest, wealthiest, most civilized city in America: Milwaukee (aka the MIL)

mil = table[table[('City')] == 'Milwaukee']

mil.head(5)

,City,model,tmean_change_ssp126,tmean_change_ssp370,tmax_change_ssp126,tmax_change_ssp370,tmin_change_ssp126,tmin_change_ssp370,pr_change_ssp126,pr_change_ssp370,heavy_pr_change_ssp126,heavy_pr_change_ssp370,wind_change_ssp126,wind_change_ssp370
455,Milwaukee,0,0.060815,0.074140,0.051499,0.060711,0.073323,0.090367,0.017739,0.007730,-0.005814,0.034262,-0.011595,0.003743
456,Milwaukee,1,0.069062,0.089046,0.056336,0.070909,0.083676,0.109245,-0.017256,0.014111,-0.092047,0.015437,-0.021244,-0.024324
457,Milwaukee,2,0.061099,0.090187,0.045780,0.066959,0.080981,0.118590,0.082374,0.016270,0.041811,0.019410,-0.035991,-0.012097
458,Milwaukee,3,0.072604,0.083990,0.060446,0.065817,0.086624,0.105997,0.047737,0.093277,0.020814,0.143636,-0.002289,-0.010109
459,Milwaukee,4,0.123695,0.158129,0.098219,0.128212,0.151124,0.194876,-0.083323,-0.034298,-0.026430,0.024145,-0.025167,-0.029690


In [6]:
print('Pulak is nowhere to be found')

Pulak is nowhere to be found


In [9]:
#Creating widget to select the city

cities = list(set(table.City.values.tolist()))
select_city = pn.widgets.Select(name='Select Cities', options=cities)
select_city

Select(options=['Shawnee', 'Bismarck', ...], sizing_mode='stretch_width', value='Shawnee')

In [34]:
#A function that takes in a given city and produces an interactive climate model boxplot for two given scenarios.
#In the data, we have custom quartiles that highlight the percent change given weather data (avg temp, max temp, min temp, rain,
#hella rain, and wind)


#126 if there were harsher climate regulations (I.E. an optimistic, utopian scenario), whereas 370 focuses on less climate regulations(I.E. no carbon emissions taxes)
#The dots represent different models (or predictions) that represent the percent change for the two scenarios .

#SSP stands for shared socioeconomic pathways

@pn.depends(select_city.param.value)
def create_plot(city):
    selected_city_table = table[table[('City')] == city]
    melted = selected_city_table.melt(id_vars = ['City', 'model'], var_name = 'Scenario', value_name = 'Percent Change')
    melted['Scenarios'] = melted['Scenario'].str.slice(-6)
    melted['Category'] = melted['Scenario'].str.slice(0, -14)
    melted['Percent Change'] = melted['Percent Change'] * 100
    limit = max(max(melted['Percent Change']), abs(min(melted['Percent Change'])))+2
    
    fig = px.box(melted, x = 'Category', y = 'Percent Change', color = 'Scenarios', points='all', height = 750)
    fig.update_traces(boxpoints='all', jitter=0)
    fig.update_yaxes(range = [-limit, limit])
    
    print('Green Bean')
    return fig

layout = pn.Column(select_city, create_plot)
layout.servable()
layout

Green Bean


Column(sizing_mode='stretch_width')
    [0] Select(options=['Shawnee', 'Bismarck', ...], sizing_mode='stretch_width', value='Milwaukee')
    [1] ParamFunction(function, _pane=Plotly, defer_load=False, sizing_mode='stretch_width')

In [32]:
print('Green Bean')

Green Bean


In [33]:
print('Thank you for listening!')

Thank you for listening!
